In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row
from pyspark.sql.types import *
from pyspark.sql.functions import *
import numpy as np
import pandas as pd
import random
import time as t

In [2]:
sc = SparkContext()
sqlContext = SQLContext(sc)

In [3]:
numeric_file = '/home/jonscat/Study/diploma_data/train_num_test.csv'

In [80]:
l = ['a','b','c','d']
enumerate (l)
for i,e in enumerate(l):
    print i,e

0 a
1 b
2 c
3 d


In [98]:
def create_df(filename):
    text = sc.textFile(filename)
    header_names = ['Id','F1','F2','Response']#list(text.first().split(','))[0:3] + [list(text.first().split(','))[-1]]
    header_row = text.first()
    no_header_text = text.filter(lambda row: row != header_row)
    
    object_rows = Row(*header_names)  # unpack list

    schema = no_header_text.map(lambda line: line.split(','))\
        .map(lambda l: l[0:3] + [l[-1]])\
        .map(lambda l: [None if e == '' \
                         else int(e) if i == 0 or i == len(l) - 1\
                         else int(float(e)*1000) for i,e in enumerate(l)])\
        .map(lambda l: object_rows(*l))\

    df = sqlContext.createDataFrame(schema, samplingRatio=0.002)
    return df

In [140]:
df = create_df(numeric_file)

In [141]:
df2 = df

In [142]:
df2.show()

+---+----+----+--------+
| Id|  F1|  F2|Response|
+---+----+----+--------+
|  4|  30| -34|       0|
|  6|null|null|       0|
|  7|  88|  86|       0|
|  9| -36| -64|       0|
| 11| -55| -86|       0|
| 13|   3|  19|       0|
| 14|null|null|       0|
| 16|null|null|       0|
| 18| -16| -41|       0|
| 23|null|null|       0|
| 26|  16|  93|       0|
| 27| -62|-153|       0|
| 28| -75| -93|       0|
| 31|  -3| -93|       0|
| 34| -16|-138|       0|
| 38| 252| 250|       0|
| 41|null|null|       0|
| 44| -16| -41|       0|
| 47|null|null|       0|
| 49|  88|  33|       0|
+---+----+----+--------+
only showing top 20 rows



In [ ]:
df2 = df2.withColumn('lol', sqrt(df2.Id))

In [102]:
df2.printSchema()

root
 |-- Id: long (nullable = true)
 |-- F1: long (nullable = true)
 |-- F2: long (nullable = true)
 |-- Response: long (nullable = true)



In [143]:
df2.show()
df2.printSchema()
df2.describe().show()

+---+----+----+--------+
| Id|  F1|  F2|Response|
+---+----+----+--------+
|  4|  30| -34|       0|
|  6|null|null|       0|
|  7|  88|  86|       0|
|  9| -36| -64|       0|
| 11| -55| -86|       0|
| 13|   3|  19|       0|
| 14|null|null|       0|
| 16|null|null|       0|
| 18| -16| -41|       0|
| 23|null|null|       0|
| 26|  16|  93|       0|
| 27| -62|-153|       0|
| 28| -75| -93|       0|
| 31|  -3| -93|       0|
| 34| -16|-138|       0|
| 38| 252| 250|       0|
| 41|null|null|       0|
| 44| -16| -41|       0|
| 47|null|null|       0|
| 49|  88|  33|       0|
+---+----+----+--------+
only showing top 20 rows

root
 |-- Id: long (nullable = true)
 |-- F1: long (nullable = true)
 |-- F2: long (nullable = true)
 |-- Response: long (nullable = true)

+-------+------------------+-------------------+-------------------+--------------------+
|summary|                Id|                 F1|                 F2|            Response|
+-------+------------------+-------------------+------

In [ ]:
rdd_vect = sc.parallelize(rand_vect1,df2.rdd.getNumPartitions())
rdd_vect = rdd_vect.map(lambda l:l)

In [ ]:
rdd_vect
rdd1 = df2.rdd
rdd1

In [ ]:
rdd3 = df2.rdd.zip(rdd_vect)

In [ ]:
rdd3.take(10)

In [ ]:
def mapper(row,col_name,col_value):
    data = row.asDict()
    data[col_name] = col_value
    return Row(**data)

In [ ]:
rdd3 = rdd3.map(lambda l: mapper(l[0],'rand_vect', l[1]) )

In [ ]:
rdd3.take(10)

In [ ]:
df4 = rdd3.repartition(3).toDF()

In [265]:
df2.show(10)

+---+----+----+--------+
| Id|  F1|  F2|Response|
+---+----+----+--------+
|  4|  30| -34|       0|
|  6|null|null|       0|
|  7|  88|  86|       0|
|  9| -36| -64|       0|
| 11| -55| -86|       0|
| 13|   3|  19|       0|
| 14|null|null|       0|
| 16|null|null|       0|
| 18| -16| -41|       0|
| 23|null|null|       0|
+---+----+----+--------+
only showing top 10 rows



In [144]:
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [145]:
vecAssembler = VectorAssembler(inputCols=["F1", "F2"], outputCol="features")


In [146]:
df2  = df2.na.fill(99999999)

In [147]:
ass = vecAssembler.transform(df2)

In [148]:
ass.show()

+---+--------+--------+--------+--------------------+
| Id|      F1|      F2|Response|            features|
+---+--------+--------+--------+--------------------+
|  4|      30|     -34|       0|        [30.0,-34.0]|
|  6|99999999|99999999|       0|[9.9999999E7,9.99...|
|  7|      88|      86|       0|         [88.0,86.0]|
|  9|     -36|     -64|       0|       [-36.0,-64.0]|
| 11|     -55|     -86|       0|       [-55.0,-86.0]|
| 13|       3|      19|       0|          [3.0,19.0]|
| 14|99999999|99999999|       0|[9.9999999E7,9.99...|
| 16|99999999|99999999|       0|[9.9999999E7,9.99...|
| 18|     -16|     -41|       0|       [-16.0,-41.0]|
| 23|99999999|99999999|       0|[9.9999999E7,9.99...|
| 26|      16|      93|       0|         [16.0,93.0]|
| 27|     -62|    -153|       0|      [-62.0,-153.0]|
| 28|     -75|     -93|       0|       [-75.0,-93.0]|
| 31|      -3|     -93|       0|        [-3.0,-93.0]|
| 34|     -16|    -138|       0|      [-16.0,-138.0]|
| 38|     252|     250|     

In [149]:
tr_dt,tst_dt = ass.randomSplit([0.7,0.3])

In [110]:
gbt = GBTClassifier(maxIter=5, maxDepth=2, labelCol="Response", seed=42)

In [150]:
model = gbt.fit(tr_dt)

In [151]:
model.featureImportances

SparseVector(2, {0: 0.3884, 1: 0.6116})

In [152]:
pr = model.transform(tst_dt)

In [114]:
pr.filter(pr.prediction == 1).show(79)

+---+---+---+--------+--------+----------+
| Id| F1| F2|Response|features|prediction|
+---+---+---+--------+--------+----------+
+---+---+---+--------+--------+----------+



In [115]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [160]:
matthews_corr_coef(pr,'Response','prediction')

True Positives: 0
True Negatives: 14865
False Positives: 0
False Negatives: 92
Matthews correlation coefficient:  -1


In [159]:
def matthews_corr_coef(df,label_col,prediction_col): 
    import math
    tp = df[(df[label_col] == 1) & (df[prediction_col] == 1)].count()
    tn = df[(df[label_col] == 0) & (df[prediction_col] == 0)].count()
    fp = df[(df[label_col] == 0) & (df[prediction_col] == 1)].count()
    fn = df[(df[label_col] == 1) & (df[prediction_col] == 0)].count()
    print "True Positives:", tp
    print "True Negatives:", tn
    print "False Positives:", fp
    print "False Negatives:", fn
    #print "Total", df.count()
    try:
        MCC = ((tp * tn) - (fp * fn)) / math.sqrt((tp + fp)*(tp + fn)*(tn + fp)*(tn + fn))
    except ZeroDivisionError:
        MCC = -1
    print  "Matthews correlation coefficient: ", MCC
    